SNOMED analytics - comparing prediabetes and diabetes patients using the Synthea synthetic dataset

In [2]:
#!pip install --user matplotlib-venn
#!pip install --user --upgrade pip
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
 

import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
from matplotlib_venn import venn3, venn3_circles

#Global Styling
plt.style.use('tableau-colorblind10')
plt.tight_layout()

In [196]:
#example uses half a million, we have ~8000 conditions for this exploration in HDA3
codeSet = pd.read_csv('~Synthea/conditions.csv', parse_dates = True)
codeSet['START'] = pd.to_datetime(codeSet['START'])
codeSet.info()
# drop columns not needed
#codeSet['Category'] = np.NaN
#some basic stats
#print('Number of unique Condition codes: '+str(codeSet['CODE'].nunique()))
#print('Number of unique patients codes: '+str(codeSet['PATIENT'].nunique()))
#print('Number of Entries: '+ str(codeSet.shape[0]))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8766 entries, 0 to 8765
Data columns (total 6 columns):
START          8766 non-null datetime64[ns]
STOP           4598 non-null object
PATIENT        8766 non-null object
ENCOUNTER      8766 non-null object
CODE           8766 non-null int64
DESCRIPTION    8766 non-null object
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 411.0+ KB


In [197]:
ObsSet = pd.read_csv('~Synthea/observations.csv')
ObsSet['DATE'] = pd.to_datetime(ObsSet['DATE'])
#Get some basic stats
print('Number of unique observations codes: '+str(ObsSet['CODE'].nunique()))
print('Number of unique patients codes: '+str(ObsSet['PATIENT'].nunique()))
print('Number of Entries: '+ str(ObsSet.shape[0]))
ObsSet.head()

Number of unique observations codes: 138
Number of unique patients codes: 1180
Number of Entries: 303875


,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,VALUE,UNITS,TYPE
0,2011-06-12,d171d808-1f31-4ad3-aba5-e03a2fa921c7,1c97b103-d387-4804-857d-5e5b719edb65,8302-2,Body Height,177.2,cm,numeric
1,2011-06-12,d171d808-1f31-4ad3-aba5-e03a2fa921c7,1c97b103-d387-4804-857d-5e5b719edb65,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,4.0,{score},numeric
2,2011-06-12,d171d808-1f31-4ad3-aba5-e03a2fa921c7,1c97b103-d387-4804-857d-5e5b719edb65,29463-7,Body Weight,79.5,kg,numeric
3,2011-06-12,d171d808-1f31-4ad3-aba5-e03a2fa921c7,1c97b103-d387-4804-857d-5e5b719edb65,39156-5,Body Mass Index,25.3,kg/m2,numeric
4,2011-06-12,d171d808-1f31-4ad3-aba5-e03a2fa921c7,1c97b103-d387-4804-857d-5e5b719edb65,8462-4,Diastolic Blood Pressure,78.1,mm[Hg],numeric


In [198]:
MedsSet = pd.read_csv('~Synthea/medications.csv',parse_dates = True)
MedsSet = MedsSet.drop(["PAYER","BASE_COST","PAYER_COVERAGE","DISPENSES","TOTALCOST"],axis=1)
#Get some basic stats
print('Number of unique Medications codes: '+str(MedsSet['CODE'].nunique()))
print('Number of unique patients codes: '+str(MedsSet['PATIENT'].nunique()))
print('Number of Entries: '+ str(MedsSet.shape[0]))
MedsSet.head()

Number of unique Medications codes: 149
Number of unique patients codes: 1106
Number of Entries: 14102


,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
0,1995-11-16,NaN,3603cd65-53a3-424b-bc82-76204326510d,f5ddf34a-85c6-4f03-9c82-34e25fde945d,999969,Amlodipine 5 MG / Hydrochlorothiazide 12.5 MG ...,59621000.0,Hypertension
1,2010-02-12,2010-02-19,3603cd65-53a3-424b-bc82-76204326510d,ae97dfe0-c619-44c1-a687-f215d7fac984,313782,Acetaminophen 325 MG Oral Tablet,10509002.0,Acute bronchitis (disorder)
2,2012-07-08,2012-07-22,3603cd65-53a3-424b-bc82-76204326510d,7616a2fe-0ddc-44b4-84ae-1bca998b1c45,562251,Amoxicillin 250 MG / Clavulanate 125 MG Oral T...,444814009.0,Viral sinusitis (disorder)
3,2011-03-26,2011-04-09,21dd496c-2fec-4c9c-a0fe-c11ad5b2cdea,86d22444-38b4-4481-bc31-8a509a56118b,309097,Cefuroxime 250 MG Oral Tablet,NaN,NaN
4,2011-03-26,2011-04-09,21dd496c-2fec-4c9c-a0fe-c11ad5b2cdea,86d22444-38b4-4481-bc31-8a509a56118b,198405,Ibuprofen 100 MG Oral Tablet,NaN,NaN


In [199]:
#example uses half a million, we have ~36000 procedures for this exploration in HDA3
ProcSet = pd.read_csv('~Synthea/procedures.csv',parse_dates = True)
ProcSet = ProcSet.drop("BASE_COST",axis=1)

ProcSet.head()
#Get some basic stats
#print('Number of unique procedure codes: '+str(ProcSet['CODE'].nunique()))
#print('Number of unique patients codes: '+str(ProcSet['PATIENT'].nunique()))
#print('Number of Entries: '+ str(ProcSet.shape[0]))

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
0,2017-06-18,d171d808-1f31-4ad3-aba5-e03a2fa921c7,b86c6047-c25b-4b9c-90df-5712852902aa,430193006,Medication Reconciliation (procedure),NaN,NaN
1,2010-02-12,3603cd65-53a3-424b-bc82-76204326510d,ae97dfe0-c619-44c1-a687-f215d7fac984,23426006,Measurement of respiratory function (procedure),10509002.0,Acute bronchitis (disorder)
2,2009-12-14,bd512b17-9e68-4b0e-8c5b-980007ecdee1,6a6caa2e-961b-432b-8b6d-6e408ef5157f,14768001,Peripheral blood smear interpretation,NaN,NaN
3,2009-12-14,bd512b17-9e68-4b0e-8c5b-980007ecdee1,6a6caa2e-961b-432b-8b6d-6e408ef5157f,415300000,Review of systems (procedure),NaN,NaN
4,2009-12-14,bd512b17-9e68-4b0e-8c5b-980007ecdee1,6a6caa2e-961b-432b-8b6d-6e408ef5157f,430193006,Medication Reconciliation (procedure),NaN,NaN


In [7]:
#!pip install --user fhir.resources
import requests #for Rest calls

#Fhir ValueSet makes parsing the json responses easier.
from fhir.resources.valueset import ValueSet 
#function ECL query -> results
def PopulateSetWithECL(ecl):
    #Compose the API call.
    endpoint= "https://ontoserver.csiro.au/stu3-latest"
    expandAPI="/ValueSet/$expand"
    sctValueSetUrl='http://snomed.info/sct?fhir_vs=ecl/'    
    urlParam={'url':sctValueSetUrl+ecl}    
    
    #Execute the Get Request, and parse the JSON response to FHIR ValueSet
    response=requests.get(endpoint+expandAPI,params=urlParam)
    j=response.json()
    vs=ValueSet(j)

    #add every code to a Set
    _set=set()
    for e in vs.expansion.contains:          
        _set.add(e.code)    
    
    return _set

Defining aggregation catagories

In [130]:
healthCategories=[
('<<106028002','Musculoskeletal problems'),
('<<106048009','Respiratory problems'),
('<<195967001','Asthma'),
('<<363346000','Cancer'),
('<<13645005','COPD'),
('<<27550009', 'Vascular disorders'),
('<<106063007','Cardiovascular problems'),
('<<249578005','Kidney problems'),
('<<74732009','Mental illness'),
('<<40733004','Infectious disease'),
('<<414022008','Blood disease'),
('<<371409005','Disorder of eye region'),
('<<75934005','Metabolic Disease'),
('<<77386006','Pregnancy')
]

diabetesCohorts=[
('<<73211009','Diabetes mellitus'),
('<<74627003', 'Diabetic complication'), 
('<<243860001', 'Diabetes finding'),
('<< 2622007|Infected ulcer of skin| OR << 95344007|Ulcer of lower limb|','Diabetic foot'),
('<<714628002|Prediabetes(finding)| OR << 15777000|Prediabetes(finding) - depreciated|','Prediabetes')
]

procedures=[
('<< 397117006|Amputation of lower limb| OR << 161622006|H/O: lower limb amputation| OR 129309007|Amputation - action|','amputation'),
('<< 781501000168105|Debridement of lower limb|','Debridement of lower limb')
]

In [200]:
#loop through the categories and label any entry that matches the healthCategory with that label.
#using simple boolean filter

for category in diabetesCohorts:
    categorySet = PopulateSetWithECL(category[0])
    #print(categorySet)
    filter = codeSet["CODE"].isin(categorySet)
    codeSet.loc[filter,"Category"] = category[1]
    
for category in healthCategories:
    categorySet = PopulateSetWithECL(category[0])
    filter = codeSet["CODE"].isin(categorySet)
    codeSet.loc[filter,"EncCategory"] = category[1]
codeSet.loc[codeSet["EncCategory"].isnull(),"EncCategory"] = "Other Condition"
codeSet.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,Category,EncCategory
0,2016-04-14,2016-04-28,d171d808-1f31-4ad3-aba5-e03a2fa921c7,0b4374e8-838d-4289-b491-f44d51360790,444814009,Viral sinusitis (disorder),NaN,Infectious disease
1,1995-08-18,NaN,3603cd65-53a3-424b-bc82-76204326510d,61f6867a-9e58-48e6-87d0-6d9495a81a79,59621000,Hypertension,NaN,Cardiovascular problems
2,2010-02-12,2010-02-19,3603cd65-53a3-424b-bc82-76204326510d,ae97dfe0-c619-44c1-a687-f215d7fac984,10509002,Acute bronchitis (disorder),NaN,Respiratory problems
3,1998-10-19,2009-10-26,bd512b17-9e68-4b0e-8c5b-980007ecdee1,e0213a36-1b17-45c9-a233-31e1042e7b28,192127007,Child attention deficit disorder,NaN,Mental illness
4,2009-12-14,NaN,bd512b17-9e68-4b0e-8c5b-980007ecdee1,31bd0967-a973-4eaf-bf40-5ca6eb341eb7,15777000,Prediabetes,Prediabetes,Other Condition


In [201]:
#dataframe filtered to see all DM / prediabetes conditions only (or neither)
DMcohort = codeSet[codeSet['Category'] == "Diabetes mellitus"]
DMpatients = DMcohort.PATIENT.unique() #array of all diabetes patients, 101
PreDcohort = codeSet[codeSet['Category'] == "Prediabetes"]
PreDpatients = PreDcohort.PATIENT.unique() #array of all prediabetes patients (inc both), 389
NoPreDMcohort = codeSet[np.isin(codeSet,PreDpatients,invert=True) & np.isin(codeSet,DMpatients,invert=True)]

#arrays of patientID: those with just both, just DM or just prediabetes 
Pre_and_DM = DMpatients[np.isin(DMpatients,PreDpatients)] # 65 
DM_only = DMpatients[np.isin(DMpatients,PreDpatients, invert=True)] #36
#len(PreDpatients[np.isin(PreDpatients, DMpatients)]) #65
Pre_only = PreDpatients[np.isin(PreDpatients, DMpatients, invert=True)] #323
NoPreDM=NoPreDMcohort.PATIENT.unique()

#ALL conditions dataframe filtered to DM / prediabetes patients
DMcodeSet = codeSet[np.isin(codeSet['PATIENT'],DMpatients)]
#DMcodeSet[['PATIENT','DESCRIPTION']].groupby(['PATIENT']).count().mean() # average number of comorbidities
PrecodeSet = codeSet[np.isin(codeSet['PATIENT'],Pre_only)]
BothcodeSet = codeSet[np.isin(codeSet['PATIENT'],Pre_and_DM)]

PreOnlycohort = PrecodeSet[PrecodeSet['Category'] == "Prediabetes"] #prediabetes condition only
Bothcohort = DMcodeSet[DMcodeSet['Category']=="Prediabetes"] #prediabetes condition only
BothDMcohort = BothcodeSet[BothcodeSet['Category']=="Diabetes mellitus"] #DM condition only

BothcodeSet["Category"]= "Prediabetes and Diabetes"
DMcodeSet["Category"] = "Diabetes"
PrecodeSet["Category"] = "Prediabetes"
#DMcodeSet[np.isin(DMcodeSet['PATIENT'],BothcodeSet['PATIENT'])].CATEGORY ="Prediabetes and Diabetes"
DMcodeSet.head()

C:\Users\williamssoph\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\williamssoph\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\williamssoph\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,Category,EncCategory
22,2001-04-06,NaN,1b37ae9f-f895-4f71-91b8-22bad1ed333a,c4eaa061-6634-4d15-b7af-dfb730a04c3e,162864005,Body mass index 30+ - obesity (finding),Diabetes,Other Condition
23,2003-04-11,NaN,1b37ae9f-f895-4f71-91b8-22bad1ed333a,8f8aa5b1-5119-42fb-898f-565fc3ac810f,44054006,Diabetes,Diabetes,Metabolic Disease
24,2003-04-11,NaN,1b37ae9f-f895-4f71-91b8-22bad1ed333a,8f8aa5b1-5119-42fb-898f-565fc3ac810f,271737000,Anemia (disorder),Diabetes,Blood disease
25,2005-04-15,NaN,1b37ae9f-f895-4f71-91b8-22bad1ed333a,8b2dddbc-6f5d-4c72-a2b0-ba4a4c185fc5,80394007,Hyperglycemia (disorder),Diabetes,Metabolic Disease
26,2005-04-15,NaN,1b37ae9f-f895-4f71-91b8-22bad1ed333a,8b2dddbc-6f5d-4c72-a2b0-ba4a4c185fc5,302870006,Hypertriglyceridemia (disorder),Diabetes,Metabolic Disease


In [13]:
#observations
DMobsSet = ObsSet[np.isin(ObsSet['PATIENT'],DMpatients)]
PreobsSet = ObsSet[np.isin(ObsSet['PATIENT'],Pre_only)]

#medications
DMmedSet = MedsSet[np.isin(MedsSet['PATIENT'],DMpatients)]
PremedSet = MedsSet[np.isin(MedsSet['PATIENT'],Pre_only)]

#procedures
DMprocSet = ProcSet[np.isin(ProcSet['PATIENT'],DMpatients)]
PreprocSet = ProcSet[np.isin(ProcSet['PATIENT'],Pre_only)]
#PreprocSet[['PATIENT','DESCRIPTION']].groupby(['PATIENT']).count().mean()

In [168]:
#Patient info
PatientsSet = pd.read_csv('~Synthea/patients.csv',usecols=['Id','BIRTHDATE','DEATHDATE','GENDER','ETHNICITY'],parse_dates = True)
PatientsSet['BIRTHDATE'] = pd.to_datetime(PatientsSet['BIRTHDATE'])
PatientsSet['DEATHDATE'] = pd.to_datetime(PatientsSet['DEATHDATE'])
t = pd.to_datetime('today')
PatientsSet['AGE'] = (t - PatientsSet['BIRTHDATE']).astype('<m8[Y]')
PatientsSet['AGE AT DEATH'] = (PatientsSet['DEATHDATE'] - PatientsSet['BIRTHDATE']).astype('<m8[Y]')
PatientsSet['AGE'] = np.where(PatientsSet['AGE AT DEATH'].notnull(), PatientsSet['AGE AT DEATH'],PatientsSet['AGE']) # change age to age at death  where apt
DMPatientsSet = PatientsSet[np.isin(PatientsSet['Id'],DMpatients)]
PrePatientsSet = PatientsSet[np.isin(PatientsSet['Id'],Pre_only)]

In [202]:
# joining columns to produce features table

DMcond = pd.merge(PatientsSet, DMcohort, how = 'right', left_on = 'Id', right_on = 'PATIENT' )
DMcond['START'] = pd.to_datetime(DMcond['START'])
DMcond['AGE DM ONSET'] = (DMcond['START'] - DMcond['BIRTHDATE']).astype('<m8[Y]')
DMcond["Category"]= "Diabetes"
DMcond=DMcond.drop(["Id","STOP","CODE","DESCRIPTION","EncCategory"],axis=1)

Precond = pd.merge(PatientsSet, PreOnlycohort, how = 'right', left_on = 'Id', right_on = 'PATIENT' )
Precond['START'] = pd.to_datetime(Precond['START'])
Precond['AGE PreD ONSET'] = (Precond['START'] - Precond['BIRTHDATE']).astype('<m8[Y]')
Precond["Category"]= "Prediabetes"
Precond=Precond.drop(["Id","STOP","CODE","DESCRIPTION","EncCategory"],axis=1)

#boths..
DM_Precond = pd.merge(PatientsSet, Bothcohort, how = 'right', left_on = 'Id', right_on = 'PATIENT' )
DM_Precond['START'] = pd.to_datetime(DM_Precond['START'])
DM_Precond['AGE PreD ONSET'] = (DM_Precond['START'] - DM_Precond['BIRTHDATE']).astype('<m8[Y]')

Pre_DM = pd.merge(PatientsSet, BothDMcohort, how = 'right', left_on = 'Id', right_on = 'PATIENT' )
Pre_DM['START'] = pd.to_datetime(Pre_DM['START'])
Pre_DM['AGE DM ONSET'] = (Pre_DM['START'] - Pre_DM['BIRTHDATE']).astype('<m8[Y]') 
#15 columns (both is 21)

# latency between prediabetes- diabetes onset for those in 'both'
BOTH = pd.merge(DM_Precond[['PATIENT','BIRTHDATE','DEATHDATE','ETHNICITY','GENDER','AGE','AGE AT DEATH','ENCOUNTER','START','AGE PreD ONSET']],Pre_DM[['PATIENT','ENCOUNTER','START','AGE DM ONSET']], how='inner',left_on = 'PATIENT', right_on='PATIENT', suffixes=('_PreD', '_DM'))
BOTH['DISEASE LATENCY'] = (BOTH['START_DM'] - BOTH['START_PreD']).astype('<m8[M]')
BOTH["Category"]= "Prediabetes and Diabetes"

#merges for stuctured table with demog, PreD and DM info in
BOTHDM = pd.merge(BOTH,DMcond,how='outer', suffixes=('_BOTH','_DM'))
BOTHDM["START_DM"]=BOTHDM["START_DM"].fillna(BOTHDM["START"])
BOTHDM["ENCOUNTER_DM"]=BOTHDM["ENCOUNTER_DM"].fillna(BOTHDM["ENCOUNTER"])
BOTHDM = BOTHDM.drop(["ENCOUNTER", "START"],axis=1)
BOTHDMPRE = pd.merge(BOTHDM,Precond,how='outer',suffixes=('_BOTH','_DM'))
BOTHDMPRE["START_PreD"]=BOTHDMPRE["START_PreD"].fillna(BOTHDMPRE["START"])
BOTHDMPRE["ENCOUNTER_PreD"]=BOTHDMPRE["ENCOUNTER_PreD"].fillna(BOTHDMPRE["ENCOUNTER"])
BOTHDMPRE = BOTHDMPRE.drop(["ENCOUNTER", "START"],axis=1)
BOTHDMPRE.head()

,PATIENT,BIRTHDATE,DEATHDATE,ETHNICITY,GENDER,AGE,AGE AT DEATH,ENCOUNTER_PreD,START_PreD,AGE PreD ONSET,ENCOUNTER_DM,START_DM,AGE DM ONSET,DISEASE LATENCY,Category
0,1b37ae9f-f895-4f71-91b8-22bad1ed333a,1961-03-31,NaT,irish,M,58.0,NaN,8b2dddbc-6f5d-4c72-a2b0-ba4a4c185fc5,2005-04-15,44.0,8f8aa5b1-5119-42fb-898f-565fc3ac810f,2003-04-11,42.0,-25.0,Prediabetes and Diabetes
1,4a56070d-d482-4854-a1fb-8a0aac88b889,1957-08-29,2010-09-16,italian,F,53.0,53.0,e02099d4-0fbd-4467-a474-d1c3fc1912a0,2008-09-04,51.0,cbe629ee-bb81-40b9-8638-6007420d9ae3,2007-08-30,50.0,-13.0,Prediabetes and Diabetes
2,b003aafe-cb06-47fd-bd4f-f8044b64b596,1964-02-29,NaT,american,M,55.0,NaN,54bf687a-9d9a-4446-9ea5-6fdf9219040a,2012-03-24,48.0,d991e87f-3e98-481a-9a93-9337c3e3d20a,2010-03-20,46.0,-25.0,Prediabetes and Diabetes
3,2c405456-ee6d-475c-b0fe-e8675589845c,1966-08-13,NaT,german,F,53.0,NaN,544635fb-4425-4711-acb3-d12d168da199,2019-08-31,53.0,bae00fee-1d7f-4b86-b831-7d559dda2e25,2012-09-01,46.0,-84.0,Prediabetes and Diabetes
4,6343cfd0-d47f-4a20-b561-b33d6441f372,1957-09-14,NaT,german,M,62.0,NaN,2f15c522-1f28-48a5-9d45-c5b08887cb12,1994-12-03,37.0,e88e5e4f-f613-4849-a938-0f5c976bc1b2,1997-09-20,40.0,33.0,Prediabetes and Diabetes


In [216]:
# join to Pre-DM onset date to split by pre-d + 5 years and pre-d - DM 5 years
#DM = BOTHDMPRE[(BOTHDMPRE['DISEASE LATENCY'] >= 0) & (BOTHDMPRE['DISEASE LATENCY'] < 60)].nunique() #27, 29 in +ve numbers
#noDM = BOTHDMPRE[(BOTHDMPRE['START_PreD'] < '2017') & (BOTHDMPRE['Category'] == "Prediabetes")].nunique() #258
CodedEnc= pd.merge(codeSet[["START","STOP","PATIENT","DESCRIPTION","EncCategory"]],BOTHDMPRE, how='inner', on='PATIENT')

# encounter data sets (eg removing encounters after DM onset)
DM = CodedEnc[(CodedEnc['DISEASE LATENCY'] >= 0) & (CodedEnc['DISEASE LATENCY'] <= 60) & (CodedEnc['START_DM'] < CodedEnc['START'])]
noDM=CodedEnc[(CodedEnc['START_PreD'] < '2017') & (CodedEnc['Category'] == "Prediabetes")]


encType_DM = DM[['PATIENT','DESCRIPTION','EncCategory']].groupby(['PATIENT',"EncCategory"]).count()
encType_DM = encType_DM.unstack(level=-1).fillna('0')
encType_noDM = noDM[['PATIENT','DESCRIPTION','EncCategory']].groupby(['PATIENT',"EncCategory"]).count()
encType_noDM = encType_noDM.unstack(level=-1).fillna('0')
encType_DM
#CodedEnc['FiveYr_PreD?'] = (CodedEnc['START'] - CodedEnc['START_PreD']).astype('<m8[M]')
#encounters_within5YPD = CodedEnc[(CodedEnc['FiveYr_PreD?'] >= 0)&(CodedEnc['FiveYr_PreD?'] <= 60)]
#encounters_afterDM = CodedEnc[(CodedEnc['Before DM?'] < 0)&((CodedEnc["Category"] == "Diabetes")|(CodedEnc["Category"] == "Prediabetes and Diabetes"))]

#encounters_afterDM[(encounters_afterDM['EncCategory'] == "Other Condition")]

DESCRIPTION         \
EncCategory                          Blood disease Cancer   
PATIENT                                                     
01cb4df5-2600-456d-9b29-767fd4f0f7d8             0      0   
0979f4fe-08c5-414e-ba1c-6ccf852bcce4             0      0   
0ddd0f9a-f593-4987-93a8-5a53162aa3ec             0      1   
1b5a3b41-6c4b-40f6-86b0-32d19b2501f5             0      0   
24b1778f-54bf-4d39-b6b7-6cc780bd3f24             0      0   
28ea39f3-a07b-457b-b12d-e35c8bd2d1dd             0      0   
2c57a897-8381-44a6-920f-e074fa6f74cf             0      2   
2ce7d774-4eee-4ae2-9f2a-59787602e693             1      0   
3a24d9f2-1389-4e32-b002-d71762ef4435             0      0   
43f14c01-8465-42ae-a884-993dce7002a4             0      0   
4b8ca9cd-6ba9-4175-ab43-a5a07f105afd             0      0   
558cd80c-07df-4207-baa1-5110400a4a31             0      0   
563383fc-a421-40ba-93f4-911eb946fd32             1      0   
5f120ce1-8fd8-4257-83e5-5438b4ad5ce7             0      0   
6343cfd0-d47f-4a20-b561-b33d6441f372             1      0   
6941e901-70f8-4b4d-b2d7-e716be3e1241             0      0   
91e630a6-05e9-4ada-9a8e-7d2d1eb22d82             0      1   
a3977ce3-93d5-41bb-a544-c93cebcbec29             0      0   
b206d59e-5523-435d-8961-555907e6d3ab             0      0   
b3e1def9-d4cf-4c54-a15a-38e936f06235             0      2   
c85e59cb-50b9-4ae5-9961-12a5008e640a             0      0   
d2578c4f-ad10-41b2-b9b5-6c1b6f096b0e             0      0   
d7466dac-6419-45f8-a609-6af519c76583             0      0   
da24e588-37fe-47aa-9780-4d154d84df5c             0      0   
e52dd621-0835-4773-a309-6bdc64e77753             0      0   
ff76c6f0-d0da-438d-bce4-0c4e88fbc752             0      0   

                                                              \
EncCategory                          Cardiovascular problems   
PATIENT                                                        
01cb4df5-2600-456d-9b29-767fd4f0f7d8                       2   
0979f4fe-08c5-414e-ba1c-6ccf852bcce4                       2   
0ddd0f9a-f593-4987-93a8-5a53162aa3ec                       0   
1b5a3b41-6c4b-40f6-86b0-32d19b2501f5                       2   
24b1778f-54bf-4d39-b6b7-6cc780bd3f24                       1   
28ea39f3-a07b-457b-b12d-e35c8bd2d1dd                       0   
2c57a897-8381-44a6-920f-e074fa6f74cf                       1   
2ce7d774-4eee-4ae2-9f2a-59787602e693                       0   
3a24d9f2-1389-4e32-b002-d71762ef4435                       1   
43f14c01-8465-42ae-a884-993dce7002a4                       1   
4b8ca9cd-6ba9-4175-ab43-a5a07f105afd                       3   
558cd80c-07df-4207-baa1-5110400a4a31                       0   
563383fc-a421-40ba-93f4-911eb946fd32                       0   
5f120ce1-8fd8-4257-83e5-5438b4ad5ce7                       2   
6343cfd0-d47f-4a20-b561-b33d6441f372                       0   
6941e901-70f8-4b4d-b2d7-e716be3e1241                       0   
91e630a6-05e9-4ada-9a8e-7d2d1eb22d82                       0   
a3977ce3-93d5-41bb-a544-c93cebcbec29                       0   
b206d59e-5523-435d-8961-555907e6d3ab                       0   
b3e1def9-d4cf-4c54-a15a-38e936f06235                       1   
c85e59cb-50b9-4ae5-9961-12a5008e640a                       0   
d2578c4f-ad10-41b2-b9b5-6c1b6f096b0e                       1   
d7466dac-6419-45f8-a609-6af519c76583                       0   
da24e588-37fe-47aa-9780-4d154d84df5c                       0   
e52dd621-0835-4773-a309-6bdc64e77753                       0   
ff76c6f0-d0da-438d-bce4-0c4e88fbc752                       0   

                                                                         \
EncCategory                          Infectious disease Kidney problems   
PATIENT                                                                   
01cb4df5-2600-456d-9b29-767fd4f0f7d8                  0               0   
0979f4fe-08c5-414e-ba1c-6ccf852bcce4                  1               1   
0ddd0f9a-f593-4987-93a8-5a53162aa3ec 

In [193]:
# join to DM onset date to split by pre-dm and post.. (12 months before vs earlier)

#CodedEnc['Before DM?'] = (CodedEnc['START_DM'] - CodedEnc['START']).astype('<m8[M]')
#encounters_beforeDM = CodedEnc[(CodedEnc['Before DM?'] >= 0)&((CodedEnc["Category"] == "Diabetes")|(CodedEnc["Category"] == "Prediabetes and Diabetes"))]
#encounters_afterDM = CodedEnc[(CodedEnc['Before DM?'] < 0)&((CodedEnc["Category"] == "Diabetes")|(CodedEnc["Category"] == "Prediabetes and Diabetes"))]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5308 entries, 0 to 5307
Data columns (total 19 columns):
START              5308 non-null object
STOP               1879 non-null object
PATIENT            5308 non-null object
DESCRIPTION        5308 non-null object
EncCategory        5308 non-null object
BIRTHDATE          5308 non-null datetime64[ns]
DEATHDATE          1697 non-null datetime64[ns]
ETHNICITY          5308 non-null object
GENDER             5308 non-null object
AGE                5308 non-null float64
AGE AT DEATH       1697 non-null float64
ENCOUNTER_PreD     3831 non-null object
START_PreD         3831 non-null datetime64[ns]
AGE PreD ONSET     3831 non-null float64
ENCOUNTER_DM       2512 non-null object
START_DM           2512 non-null datetime64[ns]
AGE DM ONSET       2512 non-null float64
DISEASE LATENCY    1035 non-null float64
Category           5308 non-null object
dtypes: datetime64[ns](4), float64(5), object(10)
memory usage: 829.4+ KB


In [206]:
# join to DM onset date to split by pre-dm and post.. (12 months before vs earlier)
CodedEnc['Before PreD?'] = (CodedEnc['START_PreD'] - CodedEnc['START']).astype('<m8[M]')
#encounters_beforeDM = CodedEnc[(CodedEnc['Before DM?'] >= 0)&((CodedEnc["Category"] == "Diabetes")|(CodedEnc["Category"] == "Prediabetes and Diabetes"))]
#encounters_afterDM = CodedEnc[(CodedEnc['Before DM?'] < 0)&((CodedEnc["Category"] == "Diabetes")|(CodedEnc["Category"] == "Prediabetes and Diabetes"))]

#encounters_afterDM[(encounters_afterDM['EncCategory'] == "Other Condition")]


In [134]:
# join to DM onset date to split by pre-dm and post.. (12 months before vs earlier)
encType_afterDM = encounters_afterDM[['PATIENT','DESCRIPTION','EncCategory']].groupby(['PATIENT',"EncCategory"]).count()
encType_afterDM = encType_afterDM.unstack(level=-1).fillna('0')
encType_beforeDM = encounters_beforeDM[['PATIENT','DESCRIPTION','EncCategory']].groupby(['PATIENT',"EncCategory"]).count()
encType_beforeDM = encType_beforeDM.unstack(level=-1).fillna('0')

encType_beforeDM
enc_all = pd.merge(BOTHDMPRE,encType_beforeDM,how='outer',suffixes=('_BOTH','_DM'))

DESCRIPTION       \
EncCategory                          Blood disease COPD   
PATIENT                                                   
01b94ddd-8997-4ee0-9a7d-73a240963755             0    0   
01cb4df5-2600-456d-9b29-767fd4f0f7d8             2    0   
0979f4fe-08c5-414e-ba1c-6ccf852bcce4             2    0   
0c671dba-1171-48f8-935b-44d85db8f457             0    0   
0ca76f6e-f990-4405-a1b5-215dc2a8735f             0    0   
0ddd0f9a-f593-4987-93a8-5a53162aa3ec             2    0   
0fbf1455-607c-44c9-9183-4259ba00222e             0    0   
12bb3d96-9c99-45d2-b3c0-df943405bfd3             2    0   
164d38cf-ffa2-4cb9-b85a-e1ebaf12fbec             1    0   
185d26ad-fb9f-40ae-afb0-94d72827d887             0    0   
1b37ae9f-f895-4f71-91b8-22bad1ed333a             2    0   
1b5a3b41-6c4b-40f6-86b0-32d19b2501f5             0    0   
1f2a4648-c23f-43f3-a468-c36ab6028234             0    0   
24b1778f-54bf-4d39-b6b7-6cc780bd3f24             2    0   
28ea39f3-a07b-457b-b12d-e35c8bd2d1dd             2    0   
2c405456-ee6d-475c-b0fe-e8675589845c             2    0   
2c57a897-8381-44a6-920f-e074fa6f74cf             2    0   
2ce7d774-4eee-4ae2-9f2a-59787602e693             0    0   
2ea28da9-0367-4304-bafa-63c472340347             0    0   
2f88c354-3ca7-487a-9718-c1fd8709a7a4             1    0   
3083b891-2c3d-4c74-bd61-ed13165fab16             1    0   
324a30f9-5353-4492-887a-21a7c79b9450             2    0   
39e8eeee-08d6-4542-b935-5924618c81cc             2    0   
3a092106-64e5-4b2f-add3-b6399e589052             1    0   
3a24d9f2-1389-4e32-b002-d71762ef4435             0    0   
3bf1dbdb-6332-4bf3-949f-77c9c2c5b9f8             0    0   
3e671c1f-bad8-430c-937b-ac703d07c9ac             0    0   
400e3e4e-4e04-4d6c-b153-0bc2ead58ed6             2    0   
40c05174-fdce-48b6-af2c-3df27de76ce1             2    0   
43dfc3fc-6fe9-410a-89ba-593861a6238e             1    0   
...                                            ...  ...   
a2854ec6-28be-48cd-b0db-4d4398520c51             0    0   
a3977ce3-93d5-41bb-a544-c93cebcbec29             0    0   
ad7f38c5-8a53-4a6c-9b6d-7ad0637164d7             0    0   
ae6669f0-0200-4f8d-83cc-d977041acad5             0    0   
b003aafe-cb06-47fd-bd4f-f8044b64b596             0    0   
b206d59e-5523-435d-8961-555907e6d3ab             0    0   
b3e1def9-d4cf-4c54-a15a-38e936f06235             2    0   
bd2a1c40-c248-4ee4-a2d1-bfa8f0cfd0f1             1    0   
bd58d43b-a597-409c-82b1-89db75ca275a             0    0   
bde71dae-bb2d-451f-b845-c260e69abdd6             2    0   
be378a4f-6973-4d3c-b6bf-294567a77933             0    0   
c71e1837-f1fb-4ac4-a05b-216cccfb0a9d             0    0   
c729d477-421d-4167-a946-ca33895605e8             0    0   
c85e59cb-50b9-4ae5-9961-12a5008e640a             0    0   
cc257fb6-c91f-46b6-80cb-d9a9a3fc9dd7             2    0   
d05cb90a-c2fc-4da3-ac6b-aded96bb48d5             2    0   
d2578c4f-ad10-41b2-b9b5-6c1b6f096b0e             2    0   
d36b57d2-052b-4b7a-9978-d4bac3f59c36             1    0   
d7466dac-6419-45f8-a609-6af519c76583             2    0   
d785bfcb-1835-42b0-9fde-18bbeba7a769             2    0   
da24e588-37fe-47aa-9780-4d154d84df5c             2    0   
da6eed88-4547-4ef0-9307-16e1fd770027             0    0   
dc3a5f43-2aba-46d2-8d06-aae7c7a44671             0    0   
e2b0f8da-4986-4f32-9030-d4c6bf5bc935             0    0   
e52dd621-0835-4773-a309-6bdc64e77753             2    0   
efe123ac-4ce6-4927-a080-0bb510aed495             0    0   
f539b505-71fd-4edd-ba65-ee7bc2d85c52             1    0   
fb68b191-6b88-49ff-b589-5b5a57050700             1    0   
fbf7da40-a027-4c60-a872-e04d718bb505             1    0   
ff76c6f0-d0da-438d-bce4-0c4e88fbc752             2    0   

                                                              \
EncCategory                          Cardiovascular problems   
PATIENT                                                        
01b94ddd-8997-4ee0-9a7d-73a240963755                       2   
01cb4d

In [62]:
# Plot HbA1C values per 'BOTH' individual
bothHbAC1Obs = ObsSet[np.isin(ObsSet['PATIENT'],Pre_and_DM) & (ObsSet.CODE == "4548-4")]
bothHbAC1Obs['VALUE'] = bothHbAC1Obs.VALUE.astype(float)
preHbAC1Obs = ObsSet[np.isin(ObsSet['PATIENT'],Pre_only) & (ObsSet.CODE == "4548-4")]
preHbAC1Obs['VALUE'] = preHbAC1Obs.VALUE.astype(float)
DMHbAC1Obs = ObsSet[np.isin(ObsSet['PATIENT'],DMpatients) & (ObsSet.CODE == "4548-4")]
DMHbAC1Obs['VALUE'] = DMHbAC1Obs.VALUE.astype(float)
NoHbAC1Obs = ObsSet[np.isin(ObsSet['PATIENT'],NoPreDM) & (ObsSet.CODE == "4548-4")]
NoHbAC1Obs['VALUE'] = NoHbAC1Obs.VALUE.astype(float)
#DMObs = ObsSet[np.isin(ObsSet['PATIENT'],DMpatients)]
#HbA1cDM = DMObs[DMObs['CODE'] == "4548-4"] 
#NoHbAC1Obs['PATIENT'].nunique() # all DM + preDM patients have HbA1c, 424/1153 non DM have

#NoHbAC1Obs[['PATIENT','VALUE']].groupby(['PATIENT']).mean().mean() # av HbA1C encounters per DM person
#DMHbAC1Obs.groupby(['PATIENT']).mean().std() # av HbA1C encounters per DM person

C:\Users\williamssoph\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\williamssoph\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\williamssoph\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [66]:
NoHbAC1Obs[(NoHbAC1Obs['VALUE']>7)].nunique() # no prediabetes only patients with HbA1c >7, 24

DATE           45
PATIENT        24
ENCOUNTER      46
CODE            1
DESCRIPTION     1
VALUE           7
UNITS           1
TYPE            1
dtype: int64

In [68]:
#date of latest observations per patient
#ObsSet.loc[ObsSet.groupby('PATIENT').DATE.idxmax(), :] #need to do pd.as date thing
#SMOKING STATUS "72166-2" and housing status "71802-3" and BMI (%) "59576-9"
#ObsSet[np.isin(ObsSet['PATIENT'],NoPreDM) & (ObsSet.CODE == "72166-2")].VALUE.describe()
#ObsSet[np.isin(ObsSet['PATIENT'],Pre_only) & (ObsSet.CODE == "72166-2")].groupby(['PATIENT']).VALUE.describe()

#label smoking status
smo = {"Never smoker": 0,"Former smoker": 1, "Current every day smoker": 2}
ObsSet = ObsSet.replace({"VALUE": smo})

#ObsSet["smoker"]= np.where(ObsSet['VALUE'] == "Current every day smoker", 1,0) #adding markers (not right)
ObsSet[ObsSet['CODE'] == "72166-2"].filter(items = ['PATIENT','VALUE']).head()
#DMHbAC1Obs[['PATIENT','ENCOUNTER']].groupby(['PATIENT']).nunique().mean()

,PATIENT,VALUE
17,d171d808-1f31-4ad3-aba5-e03a2fa921c7,0
28,d171d808-1f31-4ad3-aba5-e03a2fa921c7,0
50,d171d808-1f31-4ad3-aba5-e03a2fa921c7,0
91,bd512b17-9e68-4b0e-8c5b-980007ecdee1,1
115,21dd496c-2fec-4c9c-a0fe-c11ad5b2cdea,0


In [ ]:
BMIObs=ObsSet[np.isin(ObsSet['PATIENT'],Pre_only)&(ObsSet.CODE == "59576-9")]
BMIObs['VALUE'] = BMIObs.VALUE.astype(float)
BMIObs.groupby('PATIENT').plot(x='DATE', y='VALUE')


#only social factors conditions/obs
#medications

In [ ]:
#categorySet = PopulateSetWithECL('<< 397117006|Amputation of lower limb| OR << 161622006|H/O: lower limb amputation| OR 129309007|Amputation - action|')
#categorySet
#ProcSet[ProcSet["CODE"].isin(categorySet)] # no matches to amputation